In [13]:
import sys

sys.setrecursionlimit(20000)

# ===================================================================
# === VERSÃO 1 - RECURSIVA COM MEMORIZAÇÃO (TOP-DOWN)
# ===================================================================

def _resolver_consumo_recursivo(tamanhos_pacotes_reagentes, consumo_restante, cache_calculo):
    
    if consumo_restante < 0:
        return float('inf')
    
    if consumo_restante == 0:
        return 0
    
    if consumo_restante in cache_calculo:
        contagem_salva, _ = cache_calculo[consumo_restante]
        return contagem_salva

    melhor_cont = float('inf')
    melhor_pacote_usado = 0

    for tamanho in tamanhos_pacotes_reagentes:
        resultado_memo = _resolver_consumo_recursivo(tamanhos_pacotes_reagentes, consumo_restante - tamanho, cache_calculo)
        
        if resultado_memo != float('inf'):
            contagem_atual = 1 + resultado_memo
            
            if contagem_atual < melhor_cont:
                melhor_cont = contagem_atual
                melhor_pacote_usado = tamanho

    cache_calculo[consumo_restante] = (melhor_cont, melhor_pacote_usado)
    return melhor_cont

def calcular_minimo_pacotes_recursivo(tamanhos_pacotes_reagentes, consumo_total_necessario):
    
    cache_calculo_novo = {} 
    
    resultado = _resolver_consumo_recursivo(
        tamanhos_pacotes_reagentes, 
        consumo_total_necessario, 
        cache_calculo_novo
    )
    
    pacotes_selecionados = {}
    
    if resultado != float('inf'):
        consumo_atual = consumo_total_necessario
        while consumo_atual > 0 and consumo_atual in cache_calculo_novo:
            _, pacote_usado = cache_calculo_novo[consumo_atual]
            
            if pacote_usado == 0:
                break
                
            pacotes_selecionados[pacote_usado] = pacotes_selecionados.get(pacote_usado, 0) + 1
            consumo_atual -= pacote_usado
            
        return resultado, pacotes_selecionados
    else:
        return "Impossivel suprir essa demanda!", {}

# ===================================================================
# === FIM DA VERSÃO RECURSIVA COM MEMORIZAÇÃO 
# ===================================================================


# ===================================================================
# === VERSÃO 2 - ITERATIVA (BOTTOM-UP)
# ===================================================================

def calcular_minimo_pacotes_iterativo(tamanhos_pacotes_reagentes, consumo_total_necessario):
    
    pacotes_minimos_para_consumo = [float('inf')] * (consumo_total_necessario + 1)
    ultimo_pacote_usado = [0] * (consumo_total_necessario + 1)
    
    pacotes_minimos_para_consumo[0] = 0

    for consumo_atual in range(1, consumo_total_necessario + 1):
        for tamanho in tamanhos_pacotes_reagentes:
            if tamanho <= consumo_atual:
                
                consumo_anterior = consumo_atual - tamanho
                
                if pacotes_minimos_para_consumo[consumo_anterior] != float('inf'):
                    
                    contagem_prop = 1 + pacotes_minimos_para_consumo[consumo_anterior]
                    
                    if contagem_prop < pacotes_minimos_para_consumo[consumo_atual]:
                        pacotes_minimos_para_consumo[consumo_atual] = contagem_prop
                        ultimo_pacote_usado[consumo_atual] = tamanho
    
    resultado_final = pacotes_minimos_para_consumo[consumo_total_necessario]
    
    pacotes_selecionados = {}
    
    if resultado_final != float('inf'):
        consumo_atual = consumo_total_necessario
        while consumo_atual > 0:
            pacote = ultimo_pacote_usado[consumo_atual]
            if pacote == 0: # Segurança caso chegue em um estado inválido
                 break
            pacotes_selecionados[pacote] = pacotes_selecionados.get(pacote, 0) + 1
            consumo_atual -= pacote
            
        return resultado_final, pacotes_selecionados
    else:
        return "Impossivel suprir essa demanda!", {}

# ===================================================================
# === FIM DA VERSÃO ITERATIVA (BOTTOM-UP)
# ===================================================================

def formatar_saida_resultado(resultado, pacotes_usados):
    if isinstance(resultado, str):
        return resultado
    
    detalhes_pacotes = []
    for tamanho in sorted(pacotes_usados.keys()):
        qtde = pacotes_usados[tamanho]
        detalhes_pacotes.append(f"{qtde} pacote(s) de {tamanho}")
    
    detalhes_str = ", ".join(detalhes_pacotes)
    
    return f"{resultado} pacotes. (Detalhe: {detalhes_str})"

print("="*50)
print(" Simulação de controle de estoque da DASA ")
print("="*50)

pacotes_disponiveis_cenario_1 = [1, 5, 10, 25] 
demanda_unidades_cenario_1 = 78

print(f"\nCenário 1: Demanda de {demanda_unidades_cenario_1} unidades.")
print(f"Pacotes disponíveis: {pacotes_disponiveis_cenario_1}")

resultado_recursivo_1, pacotes_rec_1 = calcular_minimo_pacotes_recursivo(pacotes_disponiveis_cenario_1, demanda_unidades_cenario_1)
print(f"Resultado (Recursivo com Memorização): {formatar_saida_resultado(resultado_recursivo_1, pacotes_rec_1)}")

resultado_iterativo_1, pacotes_it_1 = calcular_minimo_pacotes_iterativo(pacotes_disponiveis_cenario_1, demanda_unidades_cenario_1)
print(f"Resultado (Iterativo Bottom-Up):      {formatar_saida_resultado(resultado_iterativo_1, pacotes_it_1)}")

if resultado_recursivo_1 == resultado_iterativo_1:
    print("VERIFICAÇÃO: Ambas as versões produzem o mesmo resultado. Tudo certo!")
else:
    print("ERRO: As versões produzem resultados diferentes.")


pacotes_disponiveis_cenario_2 = [3, 7, 11]
demanda_unidades_cenario_2 = 1034

print(f"\nCenário 2: Demanda grande de {demanda_unidades_cenario_2} unidades.")
print(f"Pacotes disponíveis: {pacotes_disponiveis_cenario_2}")

resultado_recursivo_2, pacotes_rec_2 = calcular_minimo_pacotes_recursivo(pacotes_disponiveis_cenario_2, demanda_unidades_cenario_2)
print(f"Resultado (Recursivo com Memorização): {formatar_saida_resultado(resultado_recursivo_2, pacotes_rec_2)}")

resultado_iterativo_2, pacotes_it_2 = calcular_minimo_pacotes_iterativo(pacotes_disponiveis_cenario_2, demanda_unidades_cenario_2)
print(f"Resultado (Iterativo Bottom-Up):      {formatar_saida_resultado(resultado_iterativo_2, pacotes_it_2)}")

if resultado_recursivo_2 == resultado_iterativo_2:
    print("VERIFICAÇÃO: Ambas as versões produzem o mesmo resultado. Tudo certo!")
else:
    print("ERRO: As versões produzem resultados diferentes.")


pacotes_disponiveis_cenario_3 = [5, 10]
demanda_unidades_cenario_3 = 17

print(f"\nCenário 3: Demanda impossível de {demanda_unidades_cenario_3} unidades.")
print(f"Pacotes disponíveis: {pacotes_disponiveis_cenario_3}")

resultado_recursivo_3, pacotes_rec_3 = calcular_minimo_pacotes_recursivo(pacotes_disponiveis_cenario_3, demanda_unidades_cenario_3)
print(f"Resultado (Recursivo com Memorização): {formatar_saida_resultado(resultado_recursivo_3, pacotes_rec_3)}")

resultado_iterativo_3, pacotes_it_3 = calcular_minimo_pacotes_iterativo(pacotes_disponiveis_cenario_3, demanda_unidades_cenario_3)
print(f"Resultado (Iterativo Bottom-Up):      {formatar_saida_resultado(resultado_iterativo_3, pacotes_it_3)}")

if resultado_recursivo_3 == resultado_iterativo_3:
    print("VERIFICAÇÃO: Ambas as versões produzem o mesmo resultado. Tudo certo!")
else:
    print("ERRO: As versões produzem resultados diferentes.")

print("\nSimulação finalizada!")

 Simulação de controle de estoque da DASA 

Cenário 1: Demanda de 78 unidades.
Pacotes disponíveis: [1, 5, 10, 25]
Resultado (Recursivo com Memorização): 6 pacotes. (Detalhe: 3 pacote(s) de 1, 3 pacote(s) de 25)
Resultado (Iterativo Bottom-Up):      6 pacotes. (Detalhe: 3 pacote(s) de 1, 3 pacote(s) de 25)
VERIFICAÇÃO: Ambas as versões produzem o mesmo resultado. Tudo certo!

Cenário 2: Demanda grande de 1034 unidades.
Pacotes disponíveis: [3, 7, 11]
Resultado (Recursivo com Memorização): 94 pacotes. (Detalhe: 94 pacote(s) de 11)
Resultado (Iterativo Bottom-Up):      94 pacotes. (Detalhe: 94 pacote(s) de 11)
VERIFICAÇÃO: Ambas as versões produzem o mesmo resultado. Tudo certo!

Cenário 3: Demanda impossível de 17 unidades.
Pacotes disponíveis: [5, 10]
Resultado (Recursivo com Memorização): Impossivel suprir essa demanda!
Resultado (Iterativo Bottom-Up):      Impossivel suprir essa demanda!
VERIFICAÇÃO: Ambas as versões produzem o mesmo resultado. Tudo certo!

Simulação finalizada!
